In [2]:
from my_libs import *
from selenium import webdriver as se
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pickle
import os

C:\Users\gli26\Anaconda2\lib\site-packages\empyrical\utils.py:32: UserWarning: Unable to import pandas_datareader. Suppressing import error and continuing. All data reading functionality will raise errors; but has been deprecated and will be removed in a later version.
  warnings.warn(msg)
C:\Users\gli26\Anaconda2\lib\site-packages\pyfolio\pos.py:28: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  ' to position notionals.'


In [5]:
sqld=connect_sql("ken",readgateway(2),host="10.0.1.17")

In [4]:
data = pd.read_csv(download_path + "finviz.csv")
data = data.drop(["No."],axis=1)

driver.get('https://finviz.com')
cookies = driver.get_cookies()

In [5]:
def parser (data):
    if data != data or data is None:
        return data
    data = data.replace("%","")
    return float(data)/100.0

for i in data.columns:
    row = 0 
    while data.loc[row,i] != data.loc[row,i] or data.loc[row,i] == None:
        row +1
    try:
        if "%" in data.loc[row,i]:
            data.loc[:,i] = data[i].apply(lambda x: parser(x))
    except:
        continue

data["Refresh_Date"] = datetime.today()   #.date()

screener = data

screener = screener.set_index("Ticker")

screener["toTarget"] = screener.Price -screener["Target Price"]

screener = screener[screener["EPS (ttm)"]>0]

#         lowerbetter = screener.groupby(["Sector"]).agg({"P/S":"rank",'P/Cash':"rank",'P/E':"rank",'P/Free Cash Flow':"rank",'Analyst Recom':"rank",'Total Debt/Equity':"rank"},ascending = True, method = "First")
lowerbetter= screener.groupby(["Sector"])[["P/S",'P/Cash','P/E','P/Free Cash Flow','Analyst Recom','Total Debt/Equity']].rank(ascending = True,method = "first")

#         higherbetter = screener.groupby(["Sector"]).agg({'Current Ratio':"rank",'Quick Ratio':"rank",'Sales growth quarter over quarter':"rank",'Performance (Week)':"rank",'Profit Margin':"rank",'EPS (ttm)':"rank",'Operating Margin':"rank",'Insider Ownership':"rank",'Institutional Ownership':"rank",'Gross Margin':"rank"},ascending = False, method = "First")

higherbetter= screener.groupby(["Sector"])[['Performance (Week)','Current Ratio','Quick Ratio','Sales growth quarter over quarter','Profit Margin','EPS (ttm)','Operating Margin','Insider Ownership','Institutional Ownership','Gross Margin']].rank(ascending = False,method = "first")

neutral = screener.groupby(["Sector"]).agg({'Average Volume':"rank",'Market Cap':"rank",'Shares Outstanding':"rank","toTarget":"rank",'Volatility (Month)':"rank",'Relative Volume':"rank"},ascending = False, method = "First")

screenermodel = screener[["Sector","Industry","Refresh_Date"]].join(lowerbetter).join(higherbetter).reset_index()

screenervol = screener[["Sector","Industry","Refresh_Date"]].join(neutral).reset_index()

In [6]:
screenervol

,Ticker,Sector,Industry,Refresh_Date,Relative Volume,Shares Outstanding,Market Cap,Average Volume,toTarget,Volatility (Month)
0,A,Healthcare,Diagnostics & Research,2021-03-14 23:13:14.074,110.5000,170.0000,169.0000,164.0000,62.0000,53.0000
1,AAME,Financial,Insurance - Life,2021-03-14 23:13:14.074,2.0000,317.0000,53.0000,799.0000,nan,859.0000
2,AAON,Industrials,Building Products & Equipment,2021-03-14 23:13:14.074,285.5000,185.0000,194.0000,107.0000,294.0000,119.0000
3,AAP,Consumer Cyclical,Specialty Retail,2021-03-14 23:13:14.074,181.5000,132.0000,178.0000,153.0000,148.0000,61.5000
4,AAPL,Technology,Consumer Electronics,2021-03-14 23:13:14.074,179.0000,304.0000,304.0000,304.0000,33.0000,69.0000
5,AAT,Real Estate,REIT - Diversified,2021-03-14 23:13:14.074,47.0000,57.0000,65.0000,50.0000,126.0000,111.0000
6,AAWW,Industrials,Airports & Air Services,2021-03-14 23:13:14.074,116.0000,101.0000,137.0000,222.0000,17.0000,286.0000
7,AB,Financial,Asset Management,2021-03-14 23:13:14.074,519.0000,674.0000,675.0000,650.0000,133.0000,402.5000
8,ABB,Industrials,Specialty Industrial Machinery,2021-03-14 23:13:14.074,90.5000,342.0000,330.0000,317.0000,183.0000,4.0000
9,ABBV,Healthcare,Drug Manufacturers - General,2021-03-14 23:13:14.074,102.5000,196.0000,200.0000,197.0000,77.0000,21.0000


In [7]:
a = get_trade_log("VIXY").iloc[-1]["TimeStamp"]

Connection Successful


In [9]:
pd.DataFrame(mongod.conn.table.find({"TimeStamp":a}))

,_id,Price,Strategy,Ticker,TimeStamp,size
0,6029df6a797a11517a0068de,17.0500,vix_dayroll,VIXY,2021-01-29 09:21:36+00:00,-52
